In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.python.framework.ops import Tensor

from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average, Maximum
from keras.losses import categorical_crossentropy
from keras.models import Model, Input, Sequential
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

from typing import Tuple, List
import glob
import os

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels = to_categorical(train_labels, num_classes=10)

In [3]:
def cifar10_cnn_model(model_input: Tensor, model_name: str) -> training.Model:
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name=model_name)
    
    return model

def make_cnn_model(train_data: np.ndarray, model_name: str):
	input_shape = train_data[0,:,:,:].shape # (32,32,3)
	model_input = Input(shape=input_shape)

	return cifar10_cnn_model(model_input, model_name)

In [4]:
NUM_EPOCHS = 15

# Note: Requires 'weights' folder to exist in order to save weights

def compile_and_train(model: training.Model, num_epochs: int, x_train: np.ndarray, y_train: np.ndarray) -> Tuple [History, str]: 
    
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto')
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train, y=y_train, batch_size=32, 
                     epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    weight_files = glob.glob(os.path.join(os.getcwd(), 'weights/*'))
    weight_file = max(weight_files, key=os.path.getctime) # most recent file
    return history, weight_file

def evaluate_error(model: training.Model) -> np.float64:
    pred = model.predict(test_images, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, test_labels)) / test_labels.shape[0]   
 
    return error

In [5]:
# Instantiate and display model 
general_classifier = make_cnn_model(train_images, 'General')
general_classifier.summary()

Model: "General"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 192)       166080    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 192)       3319

# Train model
If you want to load pre-trained weights, skip to next cell

In [18]:
history_1, classifier_weights = compile_and_train(general_classifier, NUM_EPOCHS, train_images, train_labels)

Epoch 1/15
1250/1250 [==============================] - 14s 11ms/step - loss: 1.7314 - acc: 0.3413 - val_loss: 1.4634 - val_acc: 0.4778
Epoch 2/15
1250/1250 [==============================] - 13s 11ms/step - loss: 1.2165 - acc: 0.5575 - val_loss: 1.0588 - val_acc: 0.6136
Epoch 3/15
1250/1250 [==============================] - 13s 11ms/step - loss: 0.9833 - acc: 0.6486 - val_loss: 0.8878 - val_acc: 0.6865
Epoch 4/15
1250/1250 [==============================] - 13s 11ms/step - loss: 0.8185 - acc: 0.7149 - val_loss: 0.7976 - val_acc: 0.7150
Epoch 5/15
1250/1250 [==============================] - 13s 11ms/step - loss: 0.7062 - acc: 0.7526 - val_loss: 0.7049 - val_acc: 0.7565
Epoch 6/15
1250/1250 [==============================] - 13s 11ms/step - loss: 0.6160 - acc: 0.7849 - val_loss: 0.7158 - val_acc: 0.7496
Epoch 7/15
1250/1250 [==============================] - 13s 11ms/step - loss: 0.5506 - acc: 0.8075 - val_loss: 0.6947 - val_acc: 0.7649
Epoch 8/15
1250/1250 [==========================

In [20]:
# Display name of the best weight file
classifier_weights

'/content/weights/Overall.15-0.19.hdf5'

## Load model from weights

In [8]:
WEIGHTS = os.path.join(os.getcwd(), '../weights/general', 'General.15-0.19.hdf5')
model_input = Input(shape=train_images[0,:,:,:].shape) # (32,32,3)

general_classifier = cifar10_cnn_model(model_input, 'general')
general_classifier.load_weights(WEIGHTS)

## Evaluate error rate of the classifier

In [9]:
print("Error rate - General classifier: ", evaluate_error(general_classifier)*100,'%')

Error rate - General classifier:  22.05 %
